# SpaCy 工作流

### 安装下载spaCy和必要的库

```bash
# 确保已经安装了中文模型

pip install spacy
python -m spacy download zh_core_web_sm
```

#### 调用nlp时会发生什么？

![spaCy流程将文本变为处理过的doc的图示](https://course.spacy.io/pipeline.png)

我们到现在已经写过很多遍了：给`nlp`实例传入一个文本的字符串，然后 返回一个`Doc`实例。

但`nlp`实例 *实际中* 到底做了什么？

首先，应用分词器将一段文本的字符串变成一个`Doc`实例。 然后，一系列的流程组件会依次作用在这个doc上面。 

这个例子中这些组件依次是词性标注器tagger、依存关系标注器parser、以及实体识别器entity recognizer。 

最后返回被处理过的doc，我们就可以在这个上面开展后续工作了。

In [1]:
import spacy

# 读取一个流程，创建nlp实例
nlp = spacy.load("zh_core_web_lg")

doc = nlp("我爱北京天安门")

#### 原生的流程组件

| 名字        | 描述           | 创建结果                                                  |
| ----------- | -------------- | --------------------------------------------------------- |
| **tagger**  | 词性标注器     | `Token.tag`, `Token.pos`                                  |
| **parser**  | 依存关系标注器 | `Token.dep`, `Token.head`, `Doc.sents`, `Doc.noun_chunks` |
| **ner**     | 命名实体识别器 | `Doc.ents`, `Token.ent_iob`, `Token.ent_type`             |
| **textcat** | 文本分类器     | `Doc.cats`                                                |

spaCy原生提供了很多不同的流程组件。下面是一般项目中最常用的一些组件：

词性标注器设定了`token.tag`和`token.pos`这两个属性。

依存关系标注器添加了`token.dep`和`token.head`属性，同时也负责检测句子和基础的名词 短语，也被称作名词块。

命名实体识别器将检测到的实体添加到`doc.ents`属性中，同时对词符设定了实体类别的属性， 表明该词符是否是一个实体的一部分。

最后，文本分类器设定适用于整个文本的类别，将其加入`doc.cats`属性中。

因为文本的类别往往是特定的，所以默认文本分类器不包含在任何一个训练好的流程里面。 

但我们可以用它来训练自己的系统。

#### 解构后台

![标注为zh_core_web_sm的包、文件夹、文件及config.cfg的图示](https://course.spacy.io/package_meta_zh.png)

- 流程是依次定义在模型的`config.cfg`文件里。
- 原生组件需要二进制数据来做预测。

所有我们能读进spaCy的流程包都包含了一些文件和一个 `config.cfg` (即配置文件)。

这个配置文件定义了语种和流程等等，告诉spaCy应该去初始化和配置那些组件。

原生的组件如果要做预测也要需要二进制数据。

这些数据都保存在流程包中，当我们读取流程 的时候这些数据就被读取到组件中。


#### 流程属性

我们可以使用 `nlp.pipe_names` 属性来读取当前 nlp 实例中流程组件的名字。

我们可以使用 `nlp.pipeline` 属性来读取一个组件名与组件函数构成的元组的列表。

组件函数就是那些作用在 doc 上面处理文本并设置属性的函数，比如词性标注器或者命名实体识别器。

In [2]:
# nlp.pipe_names: 流程组件名的列表

print(nlp.pipe_names)

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'ner']


In [3]:
# nlp.pipeline: (name, component)元组的列表

print(nlp.pipeline)

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec object at 0x00000262BD85EAB0>), ('tagger', <spacy.pipeline.tagger.Tagger object at 0x00000262BD85EA50>), ('parser', <spacy.pipeline.dep_parser.DependencyParser object at 0x00000262BD8B04A0>), ('attribute_ruler', <spacy.pipeline.attributeruler.AttributeRuler object at 0x00000262BD079E90>), ('ner', <spacy.pipeline.ner.EntityRecognizer object at 0x00000262BD8B0BA0>)]


### 练习

针对上面提到的几种 spaCy 工作流进行练习

#### 练习：调用NLP时会发生什么？

当我们在一个文本的字符串上面调用`nlp`时spaCy会做什么？

```python
doc = nlp("这是一个句子。")
```

1. 运行词性标注器、依存关系解析器、实体识别器然后运行分词器。
2. 对文本进行分词然后依次运行流程中的每一个组件。
3. 连接spaCy服务器计算和返回结果。
4. 初始化相应的语言，加入流程并读取模型中的二进制权重。

回答：

1. **不正确。** 

   分词器永远是在其它所有流程组件 *之前* 运行的，因为分词器会将文本的字符串转化为一个 `Doc`实例。流程也不一定需要包括词性标注器、依存关系解析器和实体识别器。

2. **正确！** 

   分词器会将文本的字符串转化为一个`Doc`实例。然后spaCy会按照顺序在文档上运行 流程中的每一个组件。

3. **不正确。** 

   spaCy的所有运算都在本机，并不需要连接任何远端服务器。

4. **不正确。** 

   当我们调用`spacy.load()`来读取流程时，spaCy会初始化相应语言，加入流程并读取 模型的二进制权重。当我们在文本上调取`nlp`之前流程就已经被读取了。


#### 练习：流程的检查

让我们一起检查一下小规模的中文流程。

- 读取`zh_core_web_sm`流程并创建`nlp`实例。
- 用`nlp.pipe_names`来打印流程组件的名字。
- 用`nlp.pipeline`来打印`(name, component)`元组的完整流程。


In [4]:
import spacy

# 读取zh_core_web_sm流程
nlp = spacy.load("zh_core_web_sm")

# 打印流程组件的名字
print(nlp.pipe_names)

# 打印完整流程的(name, component)元组
print(nlp.pipeline)

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'ner']
[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec object at 0x00000262C0AFA810>), ('tagger', <spacy.pipeline.tagger.Tagger object at 0x00000262C0AFAD50>), ('parser', <spacy.pipeline.dep_parser.DependencyParser object at 0x00000262BD8B31B0>), ('attribute_ruler', <spacy.pipeline.attributeruler.AttributeRuler object at 0x00000262C2E83190>), ('ner', <spacy.pipeline.ner.EntityRecognizer object at 0x00000262BD8B2110>)]


## 定制化流程组件

现在我们知道了spaCy的流程如何运作，我们来看看另一个非常强大的 功能：定制化流程组件。

定制化流程组件让我们可以在spaCy的流程中加入我们自己的函数，当我们在一段文本上调 用`nlp`时，这些函数就会被调用来完成比如修改doc为其增加更多数据的任务。

#### 为什么要用定制化组件？

![spaCy流程图示](https://course.spacy.io/pipeline.png)

- 使得一个函数在我们调用`nlp`时被自动执行
- 为文档document和词符token增加我们自己的元数据
- 更新一些原生的属性比如`doc.ents`


当一短文本已经被分词且`Doc`实例被创建后，流程组件会依次被应用。 spaCy支持一系列的原生组件，但也允许我们定义自己的组件。

定制化组件当我们在一段文字上调用`nlp`时会被自动执行。

当我们想要给文档和词符添加我们自己定制化的元数据时，定制化组件就尤其有用。

我们还可以用定制化组件来更新原生的属性，比如命名实体识别的结果。


#### 解构组件(1)

- 函数用来读取一个`doc`，修改和返回它。
- 用`Language.component`装饰器来注册。
- 我们可以用`nlp.add_pipe`来添加组件。



In [5]:
from spacy.language import Language

@Language.component("custom_component")
def custom_component_function(doc):
    # 对doc做一些处理
    return doc

nlp.add_pipe("custom_component")

<function __main__.custom_component_function(doc)>

#### 解构组件(2)

我们可以用下面这些关键字参数来指定在流程的 *什么位置* 添加组件：

将`last`设定为`True`会把组件加在流程的最后面。这也是默认的方法。

将`first`设定为`True`会把组件加在流程的最前面，紧紧跟在分词器之后。

`before`和`after`让我们可以定义新组件放置位置之前或者之后的已有组件名字。 比如`before="ner"`就会把新组建添加到命名实体识别器之前。

新组件位置之前或者之后的那个组件必须存在，不然spaCy就会报错。


```python
@Language.component("custom_component")
def custom_component_function(doc):
    # 对doc做一些处理
    return doc

nlp.add_pipe("custom_component")
```

| 参数     | 说明                     | 例子                                        |
| -------- | ------------------------ | ------------------------------------------- |
| `last`   | 如果为`True`则加在最后面 | `nlp.add_pipe("component", last=True)`      |
| `first`  | 如果为`True`则加在最前面 | `nlp.add_pipe("component", first=True)`     |
| `before` | 加在指定组件之前         | `nlp.add_pipe("component", before="ner")`   |
| `after`  | 加在指定组件之后         | `nlp.add_pipe("component", after="tagger")` |

#### 举例：一个简单的组件1

我们来看看一个简单的流程组件的例子。

我们从一个小的中文流程开始。

然后定义组件，也就是一个函数，读取`Doc`实例然后再把它返回出来。

我们简单把要走流程的doc的长度打印出来。

别忘了把这个doc返回出来，因为它还要被流程后面的组件处理！ 分词器创建的doc会走完全部的流程组件，所以每个组件都一定要返回其处理过的doc，这点很重要。

要让spaCy知道新的组件，我们用`@Language.component`装饰器将其注册，起名为"custom_component".

我们现在可以把组件加入到流程中了。我们设置`first=True`把它加到流程的最前面，紧跟着分词器。

我们打印流程组件名，可以看到定制化组件现在出现在起始位置。这意味着我们处理一个 doc 的时候这个组件会先被调用。


In [8]:
# 创建nlp实例
nlp = spacy.load("zh_core_web_sm")

# 定义一个定制化组件
@Language.component("custom_component")
def custom_component_function(doc):
    # 打印doc的长度
    print("Doc length:", len(doc))
    # 返回doc
    return doc

# 把组件添加到流程的最前面
nlp.add_pipe("custom_component", first=True)

# 打印流程的组件名
print("Pipeline:", nlp.pipe_names)

Pipeline: ['custom_component', 'tok2vec', 'tagger', 'parser', 'attribute_ruler', 'ner']


当我们用nlp实例处理一段文本的时候，自定义组件会被应用到doc上，打印出 这个文档的长度。

In [9]:
# 创建nlp实例
nlp = spacy.load("zh_core_web_sm")

# 定义一个定制化组件
@Language.component("custom_component")
def custom_component_function(doc):
    # 打印doc的长度
    print("Doc length:", len(doc))
    # 返回doc
    return doc

# 把组件添加到流程的最前面
nlp.add_pipe("custom_component", first=True)

# 处理一段文本
doc = nlp("这是一个句子。")

Doc length: 4


### 练习

#### 练习：定制化组件的应用场景

下面这些问题中那些可以用定制化组件来解决？请选择所有正确的答案。

1. 更新训练好的流程来改进其性能
2. 基于词符及其属性来计算我们自己定义的变量
3. 基于比如一个词典来增加新的命名实体
4. 编写对某种新语种的支持

回答：

1. **不正确。** 

   定制化组件只能修改`Doc`，并不能用来直接更新其它组件的模型权重。

2. **正确！** 

   定制化组件可以很方便地用来对documents、tokens和spans增加定制化变量。

3. **正确！** 

   定制化组件可以很方便地用来定制化`doc.ents`。

4. **不正确。** 

   定制化组件只能修改`Doc`，并不能用来直接更新其它组件的模型权重。 定制化组件只有在语言类被初始化和流程中的分词步骤结束后才能被加入到流程中， 所以定制化组件不适用于增加新的语种。


#### 练习：简单组件

这个例子中我们想要用一个定制化组件来打印文档的词符长度。

- 用`doc`长度来完成组件函数。
- 加入`"length_component"`到现有的流程中，作为其**第一个**组件。
- 试用这个新的流程，用`nlp`实例来处理一段任意的文本，比如”这是一个句子。“。

In [11]:
import spacy
from spacy.language import Language

# 定义定制化组件
@Language.component("length_component")
def length_component_function(doc):
    # 获取doc的长度
    doc_length = len(doc)
    print(f"This document is {doc_length} tokens long.")
    # 返回这个doc
    return doc


# 读取小规模的中文流程
nlp = spacy.load("zh_core_web_sm")

# 将组件加入到流程的最前面，打印流程组件名
nlp.add_pipe("length_component", first=True)
print(nlp.pipe_names)

# 处理一段文本
doc = nlp("这是一个句子。")

['length_component', 'tok2vec', 'tagger', 'parser', 'attribute_ruler', 'ner']
This document is 4 tokens long.


#### 练习：复杂组件

这个练习中我们要编写一个定制化组件，使用`PhraseMatcher`在文本中寻找动物名字， 然后把匹配到的名字加入到`doc.ents`中。我们已经在变量`matcher`中创建了含有匹配 动物名模板的`PhraseMatcher`。

- 定义这个定制化组件，在`doc`上面应用`matcher`。
- 给每一个匹配结果创建一个`Span`，添加`"ANIMAL"`的标签ID，然后 用这些新的span覆盖`doc.ents`。
- 处理文本，打印`doc.ents`中所有实体的实体文本和实体标签。

In [12]:
import spacy
from spacy.language import Language
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span

nlp = spacy.load("zh_core_web_sm")
animals = ["金毛犬", "猫", "乌龟", "老鼠"]
animal_patterns = list(nlp.pipe(animals))
print("animal_patterns:", animal_patterns)
matcher = PhraseMatcher(nlp.vocab)
matcher.add("ANIMAL", animal_patterns)

# 定义定制化组件
@Language.component("animal_component")
def animal_component_function(doc):
    # 把matcher应用到doc上
    matches = matcher(doc)
    # 为每一个匹配结果生成一个Span并赋予标签"ANIMAL"
    spans = [Span(doc, start, end, label="ANIMAL") for match_id, start, end in matches]
    # 用匹配到的span覆盖doc.ents
    doc.ents = spans
    return doc


# 把组件加入到流程中，紧跟在"ner"组件后面
nlp.add_pipe("animal_component", after="ner")
print(nlp.pipe_names)

# 处理文本，打印doc.ents的文本和标签
doc = nlp("我养了一只猫和一条金毛犬。")
print([(ent.text, ent.label_) for ent in doc.ents])

animal_patterns: [金毛犬, 猫, 乌龟, 老鼠]
['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'ner', 'animal_component']
[('猫', 'ANIMAL'), ('金毛犬', 'ANIMAL')]


> 上述就是一个完美基于规则的实体匹配流程组件。

## 扩展属性 

本节课中我们会学习如何添加定制化属性到`Doc`、`Token`和`Span`实例中 来储存一些定制化的数据。

#### 设置定制化属性

- 添加定制化元数据到文档document、词符token和跨度span中
- 通过`._`属性来读取

定制化属性可以让我们添加任何的元数据到doc、token和span中。 这些数据可以一次性添加，也可以动态被计算出来。

定制化属性通过`._`（点加下划线）属性来读取，这样我们可以很清楚看到这些属性是 被用户添加的而不是spaCy的内建属性如`token.text`。

属性需要注册在从`spacy.tokens`导入的全局`Doc`、`Token`和`Span`类上。我们已经在 前面几个章节中使用过它们。我们用`set_extension`方法将一个定制化属性注册到 `Doc`、`Token`和`Span`上。

第一个参数是属性名字。关键词参数让我们可以定义其值是如何被计算出来的。 在这个例子中这些定制化参数有默认值，也可以被覆盖重写。

In [58]:
doc._.title = "My document"
token._.is_color = True
span._.has_color = False

In [57]:
# 导入全局类
from spacy.tokens import Doc, Token, Span

# 在Doc、Token和Span上设置扩展属性
Doc.set_extension("title", default=None, force=True)
Token.set_extension("is_color", default=False, force=True)
Span.set_extension("has_color", default=False, force=True)

#### 扩展属性类别

1. 特性（Attribute）扩展
2. 属性（Property）扩展
3. 方法（Method）扩展

##### 特性（Attribute）扩展

- 设置一个可以被覆盖的默认值。


In [60]:
from spacy.tokens import Token

# 为Token设置一个有默认值的扩展
Token.set_extension("is_color", default=False, force=True)

doc = nlp("天空是蓝色的。")

# 覆盖默认扩展特性的值
doc[2]._.is_color = True

##### 属性（Property）扩展

- 设置一个取值器（getter）和一个可选的赋值器（setter）函数。
- 取值器只有当你 *提取* 属性值的时候才会被调用。


In [62]:
from spacy.tokens import Token

# 定义取值器函数
def get_is_color(token):
    colors = ["红色", "黄色", "蓝色"]
    return token.text in colors

# 为词符设置有取值器的扩展
Token.set_extension("is_color", getter=get_is_color, force=True)

doc = nlp("天空是蓝色的。")
print(doc[2]._.is_color, "-", doc[2].text)

True - 蓝色


##### 属性（Property）扩展

- `Span`扩展大部分情况下总是需要有一个取值器。



In [64]:
from spacy.tokens import Span

# 定义取值器函数
def get_has_color(span):
    colors = ["红色", "黄色", "蓝色"]
    return any(token.text in colors for token in span)

# 为Span设置一个带有取值器getter的扩展
Span.set_extension("has_color", getter=get_has_color, force=True)

doc = nlp("天空是蓝色的")
print(doc[1:4]._.has_color, "-", doc[1:4].text)
print(doc[0:2]._.has_color, "-", doc[0:2].text)

True - 是蓝色的
False - 天空是


##### 方法（Method）扩展

- 作为一个实例的方法引入一个**函数**
- 可以向扩展函数中传入**参数**


In [65]:
from spacy.tokens import Doc

# 定义含有参数的方法
def has_token(doc, token_text):
    in_doc = token_text in [token.text for token in doc]
    return in_doc

# 在doc上设置方法扩展
Doc.set_extension("has_token", method=has_token)

doc = nlp("天空是蓝色的。")
print(doc._.has_token("蓝色"), "- 蓝色")
print(doc._.has_token("云朵"), "- 云朵")

True - 蓝色
False - 云朵


### 练习

### 练习：设置扩展属性1

#### 第一部分

- 用`Token.set_extension`来注册`"is_country"`（默认是`False`）。
- 对`"Spain"`更新该扩展属性，然后对所有词符打印这个属性。


In [25]:
import spacy
from spacy.tokens import Token

nlp = spacy.load("zh_core_web_sm")

# 注册词符的扩展属性"is_country"，其默认值是False
Token.set_extension("is_country", default=False)

# 处理文本，将词符"新加坡"的is_country属性设置为True
doc = nlp("我住在新加坡。")
doc[2]._.is_country = True

# 对所有词符打印词符文本及is_country属性
print([(token.text, token._.is_country) for token in doc])

[('我', False), ('住在', False), ('新加坡', True), ('。', False)]


#### 第二部分

- 用`Token.set_extension`来注册`"reversed"`（取值函数是`get_reversed`）。
- 对所有词符打印这个属性的值。


In [26]:
import spacy
from spacy.tokens import Token

nlp = spacy.blank("zh")

# 定义取值器函数，读入一个词符并返回其逆序的文本
def get_reversed(token):
    return token.text[::-1]


# 注册词符的扩展属性get_reversed及其取值器get_reversed
Token.set_extension("reversed", getter=get_reversed)

# 处理文本，打印没一个词符的逆序属性
doc = nlp("我说的所有话都是假的，包括这一句。")
for token in doc:
    print("reversed:", token._.reversed)

reversed: 我
reversed: 说
reversed: 的
reversed: 所
reversed: 有
reversed: 话
reversed: 都
reversed: 是
reversed: 假
reversed: 的
reversed: ，
reversed: 包
reversed: 括
reversed: 这
reversed: 一
reversed: 句
reversed: 。


### 练习：设置扩展属性2

####  第一部分

- 完成`get_has_number`函数。
- 用`Doc.set_extension`来注册`"has_number"`（取值函数是`get_has_number`） 并打印这个属性的值。


In [27]:
import spacy
from spacy.tokens import Doc

nlp = spacy.blank("zh")

# 定义取值器函数
def get_has_number(doc):
    # 返回是否doc中的任一个词符的token.like_num返回True
    return any(token.like_num for token in doc)


# 注册Doc的扩展属性"has_number"及其取值器get_has_number
Doc.set_extension("has_number", getter=get_has_number)

# 处理文本，检查定制化的has_number属性
doc = nlp("这家博物馆在2012年关了五个月。")
print("has_number:", doc._.has_number)

has_number: True


#### 第二部分

- 用`Span.set_extension`来注册`"to_html"`（`to_html`方法）。
- 在`doc[0:2]`上用标签`"strong"`来调用它。


In [28]:
import spacy
from spacy.tokens import Span

nlp = spacy.blank("zh")

# 定义这个方法
def to_html(span, tag):
    # 将span文本包在HTML标签中并返回
    return f"<{tag}>{span.text}</{tag}>"


# 注册这个Span方法扩展名"to_html"及其方法to_html
Span.set_extension("to_html", method=to_html)

# 处理文本，在span上调用to_html方法及其标签名"strong"
doc = nlp("大家好，这是一个句子。")
span = doc[0:3]
print(span._.to_html("strong"))

<strong>大家好</strong>


### 练习：实体和扩展

在这个练习中，我们要结合定制化属性扩展和统计预测结果，创建一个属性取值函数，当 span为一个人、组织或者位置时返回其维基百科的查询URL。

- 完成`get_wikipedia_url`这个取值函数，使其只有在span的标签在标签列表中时 才返回URL。
- 用取值函数`get_wikipedia_url`设置`Span`的扩展`"wikipedia_url"`。
- 遍历`doc`中的实体，输出它们的维基百科URL。


In [29]:
import spacy
from spacy.tokens import Span

nlp = spacy.load("zh_core_web_sm")


def get_wikipedia_url(span):
    # 如果span有其中一个标签则获取其维基百科URL
    if span.label_ in ("PERSON", "ORG", "GPE", "LOCATION"):
        entity_text = span.text.replace(" ", "_")
        return "https://zh.wikipedia.org/w/index.php?search=" + entity_text


# 设置Span的扩展wikipedia_url及其取值器get_wikipedia_url
Span.set_extension("wikipedia_url", getter=get_wikipedia_url)

doc = nlp(
    "出道这么多年，周杰伦已经成为几代年轻人共同的偶像。"
)
for ent in doc.ents:
    # 打印实体的文本和其维基百科URL
    print(ent.text, ent._.wikipedia_url)

周杰伦 https://zh.wikipedia.org/w/index.php?search=周杰伦


### 练习：含有扩展的组件

把扩展参数和定制化流程组件结合在一起会发挥很大的作用。在这个练习中，我们要写一个 流程组件，寻找国家名和一个返回国家首都（如果存在的话）的定制化属性。

`matcher`变量中已经有一个匹配所有国家的短语匹配器。`CAPITALS`变量中则有一个把国家名 映射到其首都城市的字典。

- 完成`countries_component_function`，为所有匹配结果创建一个含有标签`"GPE"`（地理政治实体） 的`Span`。
- 把组件加入到流程中。
- 使用取值函数`get_capital`注册Span的扩展属性`"capital"`。
- 处理文本，对每一个`doc.ents`中的实体打印其实体文本、实体标签和实体的首都城市。


In [31]:
import json
import spacy
from spacy.language import Language
from spacy.tokens import Span
from spacy.matcher import PhraseMatcher

with open("countries.json", encoding="utf8") as f:
    COUNTRIES = json.loads(f.read())

with open("capitals.json", encoding="utf8") as f:
    CAPITALS = json.loads(f.read())

nlp = spacy.blank("zh")
matcher = PhraseMatcher(nlp.vocab)
matcher.add("COUNTRY", list(nlp.pipe(COUNTRIES)))


@Language.component("countries_component")
def countries_component_function(doc):
    # 对所有匹配结果创建一个标签为"GPE"的实体Span
    matches = matcher(doc)
    doc.ents = [Span(doc, start, end, label="GPE") for match_id, start, end in matches]
    return doc


# 把这个组件加入到流程中
nlp.add_pipe("countries_component")
print(nlp.pipe_names)

# 取值器，在国家首都的字典中寻找span的文本
get_capital = lambda span: CAPITALS.get(span.text)

# 用这个取值器注册Span的扩展属性"capital"
Span.set_extension("capital", getter=get_capital, force=True)

# 处理文本，打印实体文本、标签和首都属性
doc = nlp("新加坡可能会和马来西亚一起建造高铁。")
print([(ent.text, ent.label_, ent._.capital) for ent in doc.ents])

['countries_component']
[('新加坡', 'GPE', '新加坡'), ('马来西亚', 'GPE', '吉隆坡')]


## 规模化和性能

本节课中我们来学习一些技巧，让spaCy流程可以运作得尽可能快速，并且 能够高效处理大规模语料。


##### 处理大规模语料

- 使用`nlp.pipe`方法
- 用流模式来处理文本，生成`Doc`实例
- 这比直接在每段文本上面调用`nlp`快得多

如果我们要处理很多段文本然后创建一系列的`Doc`实例，使用`nlp.pipe`方法 可以极大地加速这一过程。

这个方法用流模式来处理文本生成`Doc`实例。

这种方法大大快过在每段文本上调用nlp，原因是它对目标文本集进行了打包。

`nlp.pipe`是一个产生`Doc`实例的生成器，所以要获得doc的列表记住要对其调用`list`方法。

**不好的方法：**

```python
docs = [nlp(text) for text in LOTS_OF_TEXTS]
```

**好的方法：**

```python
docs = list(nlp.pipe(LOTS_OF_TEXTS))
``` 

##### 传入语境(1)

- 在`nlp.pipe`设置`as_tuples=True`，这样我们可以传入一些列形式为 `(text, context)`的元组。
- 产生一系列`(doc, context)`元组。
- 当我们要把`doc`关联到一些元数据时这种方法就很有用。


In [32]:
data = [
    ("这是一段文本", {"id": 1, "page_number": 15}),
    ("以及另一段文本", {"id": 2, "page_number": 16}),
]

for doc, context in nlp.pipe(data, as_tuples=True):
    print(doc.text, context["page_number"])

这是一段文本 15
以及另一段文本 16


`nlp.pipe`支持传入文本/语境的元组，我们只需要设置`as_tuples`为`True`。

然后该方法会生成文档/语境的元组。

当我们想要传入新增的元数据时这就很有用，比如我们想要添加文本对应的ID或是一个页码。

##### 传入语境(2)

我们甚至可以把语境的元数据加入到定制化的属性中。

本例中我们要注册两个扩展属性，`id`和`page_number`。它们的默认值都是`None`。

处理文本和传入语境之后，我们就可以用我们的语境元数据来重写doc的扩展属性。


In [33]:
from spacy.tokens import Doc

Doc.set_extension("id", default=None)
Doc.set_extension("page_number", default=None)

data = [
    ("这是一段文本", {"id": 1, "page_number": 15}),
    ("以及另一段文本", {"id": 2, "page_number": 16}),
]

for doc, context in nlp.pipe(data, as_tuples=True):
    doc._.id = context["id"]
    doc._.page_number = context["page_number"]

##### 只用分词器(1)

另一种常见的情况是，有时候我们已经读入了一个模型来做一些其它的处理， 但是对某一个特定的文本我们只需要运行分词器。

我们没有必要跑完整个流程因为这会比较慢，我们还会拿到很多我们并不需要的 模型预测结果。

![spaCy流程图解](https://course.spacy.io/pipeline.png)

- 不要跑整个流程！

##### 只用分词器(2)

如果我们只是需要一个分词过的`Doc`实例，我们可以用`nlp.make_doc`方法 读入一段文本并返回一个doc。

这也是spaCy后台所做的事情：流程组件在被调用之前，`nlp.make_doc`会先把文本变 成一个doc。

- 用`nlp.make_doc`将一段文本变成`Doc`实例

**不好的方法：**

```python
doc = nlp("Hello world")
```

**好的方法：**

```python
doc = nlp.make_doc("Hello world!")
```

##### 关闭流程组件

spaCy允许我们暂时关闭一些流程组件，方法是用`nlp.select_pipes` 这个管理器。

这个方法需要一个关键词参数`enable`或者`disable`，可以定义一个包含了需要关闭的一个或多个流程组件的名字的列表。 比如我们只想要用实体识别器来处理文档，我们就可以暂时关闭词性标注器tagger 和依存关系标注器parser。

在`with`代码块之后，那些被关闭的流程组件会被自动重新启用。

在`with`代码块里面，spaCy只会跑未被关闭的剩余组件。

- 使用`nlp.select_pipes`来暂时关闭一个或多个流程组件。

```py
# 关闭词性标注器tagger和依存关系标注器parser
with nlp.select_pipes(disable=["tagger", "parser"]):
    # 处理文本并打印实体结果
    doc = nlp(text)
    print(doc.ents)
```

- `with`代码块之后这些组件会重新启用
- 这些组件关闭后spaCy流程只会跑剩余的未被关闭的组件

### 练习

#### 练习：处理流

在这个练习中，我们要使用`nlp.pipe`来做一些更高效的文本处理。

`nlp`实例已经为我们创建好了。在变量`TEXTS`中有一个关于流行美国快餐连锁的推特列表。


##### 第一部分

- 用`nlp.pipe`重写这个例子。不要直接遍历文本来处理它们，而是遍历`nlp.pipe`产生的 `doc`实例。


In [41]:
import json
import spacy

nlp = spacy.load("zh_core_web_sm")

with open("weibo.json", encoding="utf8") as f:
    TEXTS = json.loads(f.read())

# 处理文本，打印形容词
for doc in nlp.pipe(TEXTS):
    print([token.text for token in doc if token.pos_ == "ADJ"]) # ADJ形容词

[]
[]
[]
['老']
['帅帅']
[]


##### 第二部分

- 用`nlp.pipe`重写这个例子。记着对结果调用`list()`来把它变为一个列表。


In [50]:
import json
import spacy

nlp = spacy.load("zh_core_web_sm")

with open("weibo.json", encoding="utf8") as f:
    TEXTS = json.loads(f.read())

# 处理文本，打印实体
docs = list(nlp.pipe(TEXTS))
entities = [doc.ents for doc in docs]
print(*entities)

() (汉堡, 汉堡) () (中国, 麦当劳, 北京) (麦当劳,) (今天, 早上, 麦当劳, 一整天)


##### 第三部分

- 用`nlp.pipe`重写这个例子。记着对结果调用`list()`来把它变为一个列表。


In [43]:
import spacy

nlp = spacy.blank("zh")

people = ["周杰伦", "庞麦郎", "诸葛亮"]

# 为PhraseMatcher创建一个模板列表
patterns = list(nlp.pipe(people))

#### 练习：在语境中处理数据

在这个练习中，我们要用定制化属性将作者和书的一些信息加入到引用中。

变量`DATA`里有一个`[text, context]`的示例列表。文本text是一些有名书籍的引用， 而语境context是一些键值为`"author"`和`"book"`的字典。

- 使用`set_extension`方法在`Doc`上注册定制化属性`"author"`和`"book"`，其默认值 为`None`。
- 使用`nlp.pipe`，设置`as_tuples=True`，处理`DATA`中的`[text, context]`对。
- 使用传入的对应信息作为语境覆盖`doc._.book`和`doc._.author`。


In [49]:
import json
import spacy
from spacy.tokens import Doc

with open("bookquotes.json", encoding="utf8") as f:
    DATA = json.loads(f.read())

nlp = spacy.blank("en")

# 注册Doc的扩展"author"（默认值为None）
Doc.set_extension("author", default=None, force=True)

# 注册Doc的扩展"book"（默认值为None）
Doc.set_extension("book", default=None, force=True)

for doc, context in nlp.pipe(DATA, as_tuples=True):
    # 从context中设置属性doc._.book和doc._.author
    doc._.book = context["book"]
    doc._.author = context["author"]

    # 打印文本和定制化的属性数据
    print(f"{doc.text}\n — '{doc._.book}' by {doc._.author}\n")

One morning, when Gregor Samsa woke from troubled dreams, he found himself transformed in his bed into a horrible vermin.
 — 'Metamorphosis' by Franz Kafka

I know not all that may be coming, but be it what it will, I'll go to it laughing.
 — 'Moby-Dick or, The Whale' by Herman Melville

It was the best of times, it was the worst of times.
 — 'A Tale of Two Cities' by Charles Dickens

The only people for me are the mad ones, the ones who are mad to live, mad to talk, mad to be saved, desirous of everything at the same time, the ones who never yawn or say a commonplace thing, but burn, burn, burn like fabulous yellow roman candles exploding like spiders across the stars.
 — 'On the Road' by Jack Kerouac

It was a bright cold day in April, and the clocks were striking thirteen.
 — '1984' by George Orwell

Nowadays people know the price of everything and the value of nothing.
 — 'The Picture Of Dorian Gray' by Oscar Wilde



#### 练习：选择性处理

在这个练习中，我们使用`nlp.make_doc`和`nlp.select_pipes`方法只运行我们选择的 组件来处理文本。

##### 第一部分

- 用`nlp.make_doc`重写代码使其只对文本做分词。


In [52]:
import spacy

nlp = spacy.load("zh_core_web_sm")
text = (
    "在300多年的风雨历程中，历代同仁堂人始终恪守“炮制虽繁必不敢省人工，品味虽贵必不敢减物力”的古训，"
    "树立“修合无人见，存心有天知”的自律意识，造就了制药过程中兢兢小心、精益求精的严细精神。"
)

# 仅对文本做分词
doc = nlp.make_doc(text)
print([token.text for token in doc])

['在', '300多', '年', '的', '风雨', '历程', '中', '，', '历代', '同仁', '堂人', '始终', '恪守', '“', '炮制', '虽', '繁必', '不', '敢', '省', '人工', '，', '品味', '虽', '贵必', '不', '敢', '减物力', '”', '的', '古训', '，', '树立', '“', '修合', '无', '人', '见', '，', '存心', '有', '天知', '”', '的', '自律', '意识', '，', '造就', '了', '制药', '过程', '中', '兢兢小心', '、', '精益求精', '的', '严细', '精神', '。']


##### 第二部分

- 用`nlp.select_pipes`方法关闭词性标注(tagger)和词性还原(lemmatizer)的组件。
- 处理文本，将所有`doc`中的结果实体打印出来。


In [54]:
import spacy

nlp = spacy.load("zh_core_web_sm")
text = (
    "在300多年的风雨历程中，历代同仁堂人始终恪守“炮制虽繁必不敢省人工，品味虽贵必不敢减物力”的古训，"
    "树立“修合无人见，存心有天知”的自律意识，造就了制药过程中兢兢小心、精益求精的严细精神。"
)

# 关闭tagger和parser
with nlp.select_pipes(disable=["tagger", "parser"]):
    # 处理文本
    doc = nlp(text)
    # 打印doc中的实体
    print(doc.ents)

(300多年,)
